# Experiments with BTCR 2.0 Offline Creation

## Want to Support

* Offline DID Creation
* Resolution
* Update
* Deactivate

## Use cases

* Address only, no accompanying DID document
* Address only, requires accompanying DID document
* Batch accumulator thing
* Deterministic from onchain transactions

## TODO

* Define a JSONLD context for DID BTCR 2.0
* Add support for aggregated updated **Note: not sure how this would work with current flow**

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## Generate Mnemonic and HDPrivateKey

A HD is effectively a wallet / keyring that can be used to deterministically generate an infinite amount of public/private key pairs.

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey

mnemonic = secure_mnemonic()
hd_privatekey = HDPrivateKey.from_mnemonic(mnemonic)

In [3]:
from buidl.helper import encode_base58

# We could define a specific purpose (Current purposes are for different script types)
didkey_purpose = "11"

private_key = hd_privatekey.get_private_key(didkey_purpose, address_num=1)
public_key = private_key.point
sec_pubkey = public_key.sec()
base58_pk = encode_base58(sec_pubkey)

In [4]:
signet_address = public_key.address(network="signet")
print(signet_address)

ms88VTERNyZAuhuxzuV6jLccUVuQhTA7D3


## Define Offline DID DID document

1. Create proto DID document (no identifiers)
2. SHA256 hash proto document
3. Generate p2sh address for RedeemScript of `[OP_RETURN, PUSH_32, sha256_doc_hash]`
4. Set DID to did:btcr:2:p2sh_address
5. Populate `id` and `controller` fields in proto DID document to produce the final DID documen

### 1. Define proto DID document

This DID document can contain any verificationMethods, service endpoints etc. It will be passed along with the DID that identifies it.

In [5]:
proto_did_doc = {
    "@context": ["https://www.w3.org/ns/did/v1"],
    "verificationMethod": [
       {
            "id": "#satoshi",
           # What would we do about this aswell. Since we don't have the controller identifier
#             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
            "type": "EcdsaSecp256k1VerificationKey2019",
            "publicKeyBase58": base58_pk
        },
    ##########################      EXAMPLE       #################################
        ### What are the resolution rules that parse the Bitcoin Tx at ref xyv2-xzpq-q9wa-p7t
        ### To populate this verificationMethod into the didDocument.
        ### E.g. scriptPubKey from UTX0 at index 1 in tx at ref xyv2-xzpq-q9wa-p7t
#         {
#              "id": "did:btcr2:xyv2-xzpq-q9wa-p7t#vm-1",
#              "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
#              "type": "BIP322",
#              "address": address
#         }
    
    ###############################################################################
    ],
    "authentication": ["#satoshi"],
    "assertionMethod": ["#satoshi"],
    "serviceEndpoint": "http://satoshi_endpoint.com"
}


### 2. SHA256 hash the proto document

Note: we should probably canonicalize the proto DID doc? But currently the context does not resolve. Would it matter that this is proto rather than a full DID doc?

In [20]:
# TODO: This is currently returning NOTHING
# from pyld import jsonld
# normalized = jsonld.normalize(
#     proto_did_doc, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})

# normalized

''

In [6]:
from buidl.helper import sha256, str_to_bytes
import json

bytes_doc = str_to_bytes(json.dumps(proto_did_doc))
sha256_doc = sha256(bytes_doc)
sha256_doc

b'`2\xd5\x10&]l\xd0L\x06\x96\xfd\xa1\xd1Z=\x1e!\xf4\x07$\x97\xb6$\xcd\x8c\xf6\x18r3"e'

### 3. Generate p2sh address from doc hash

Note: This p2sh has a redeem script containing OP_RETURN, hence is unspendable

In [7]:
from buidl.script import RedeemScript
commands = [106, sha256_doc]

script = RedeemScript(commands)

In [8]:
# Notice the script_pubkey is a pay to script hash 
script.script_pubkey()

OP_HASH160 adeb819bbc44bdf7e6fa9078e9ba2753143fb35e OP_EQUAL 

In [9]:
network="signet"

p2sh_address = script.address(network)

### 4. Set DID to did:btcr:2:ps2h_address

In [10]:
offline_did = "did:btcr:2:" + p2sh_address
offline_did

'did:btcr:2:2N96q1jXR7Ksm3NKd5Yiqf35EiFuecbkr5W'

### 5. Populate id and controller fields in proto_did_doc

In [11]:
did_doc = proto_did_doc
did_doc["id"] = offline_did

for vm in did_doc["verificationMethod"]:
    vm["controller"] = offline_did
    
# Does it matter about ordering? Is this where JSON LD canonicalization comes in?
print(did_doc)

{'@context': ['https://w3id.org/did/v0.11', 'https://w3id.org/btcr/v2'], 'verificationMethod': [{'id': '#satoshi', 'type': 'EcdsaSecp256k1VerificationKey2019', 'publicKeyBase58': 'z5LBRRchkWcMDAWBrQ1N1B65VbZ28kvNmnEwNQDSEsh7', 'controller': 'did:btcr:2:2N96q1jXR7Ksm3NKd5Yiqf35EiFuecbkr5W'}], 'authentication': ['#satoshi'], 'assertionMethod': ['#satoshi'], 'serviceEndpoint': 'http://satoshi_endpoint.com', 'id': 'did:btcr:2:2N96q1jXR7Ksm3NKd5Yiqf35EiFuecbkr5W'}


## Resolve Offline DID

Will gives offline DID and associated DID document to Ryan who wishes to verify that the DID document is the correct and up to date.

1. Verify did_doc `id` equals the did
2. Check method specific identifier (p2sh_address) does not exist in TxOut on network
-- If not

3. Remove `id` and `controller` fields from did_doc
4. SHA256 hash did_doc (canonicalize it)?
5. Create p2sh with RedeemScript commands `[OP_RETURN, PUSH_32, sha256_doc_hash]`
6. Create p2sh_address and check equal to method_identifier

### 1. Verify did_doc id equals the did

In [20]:
assert(offline_did == did_doc["id"])

### 2. Check method specific identifier (p2sh_address) does not exist in TxOut on network

In [21]:
# TODO: Assume not on network for now

### 3. Remove id and controller fields from did_doc

In [22]:
no_id_diddoc = did_doc
del no_id_diddoc["id"]

for vm in no_id_diddoc["verificationMethod"]:
    del vm["controller"]
    
no_id_diddoc

{'@context': ['https://w3id.org/did/v0.11', 'https://w3id.org/btcr/v2'],
 'verificationMethod': [{'id': '#satoshi',
   'type': 'EcdsaSecp256k1VerificationKey2019',
   'publicKeyBase58': 'iqa9qrXn7FQi9GCnFL8X21fCyevWiJuPEUdKK66buGvs'}],
 'authentication': ['#satoshi'],
 'assertionMethod': ['#satoshi'],
 'serviceEndpoint': 'http://satoshi_endpoint.com'}

### 4. SHA256 hash did_doc

In [23]:
bytes_doc = str_to_bytes(json.dumps(no_id_diddoc))
sha256_doc = sha256(bytes_doc)
sha256_doc

b'\xfa\xba\x1c?\xf6q\x95\xa0\x14\xadg?\xc9\xaff\xc4\t\xecL\xf9\xe75\xe1\x9d<\x7fv|\xa4\xc0\xa4\xbd'

### 5. Create p2sh

In [34]:
# First we must determine the network of the DID

did_identifier = offline_did.split(":")[3]
deduced_network = None
if did_identifier[:1] == '3':
    print("Mainnet")
    deduced_network = "mainnet"
else:
    deduced_network = "testnet"

In [35]:
commands = [106, sha256_doc]

script = RedeemScript(commands)
p2sh_address = script.address(deduced_network)


In [36]:
p2sh_address

'2N3NU4aCsZfcdY9xBr6q7EW44i9HUMiBjWF'

### 6. Create p2sh_address and check equal to method_identifier

In [37]:
split_diddoc = offline_did.split(":")
method_identifier = split_diddoc[3]
assert(method_identifier == p2sh_address)

## Update the DID by publishing a Bitcoin Tx

- Bitcoin Tx must contain UTXO with p2sh address used as DID method specific identifier
- Can contain any number of other UTXO

### Prerequisites 

- Control of a UTXO with funds

### Questions

- Is DID doc now deterministically resolvable from onchain inf alone
    - Don't think this can be the case. How would the serviceEndpoint be retrieved?
- Is the alternative to require a continuation document ALWAYS specified using the OP_RETURN approach defined in v0.1?
    - If this is the case, then is any of the contents of the DID document determined from on chain data (e.g. other UTXO)
- Should the offline DID document contain a verificationMethod containing the public key that will be used to sign the originating btc tx containing the p2sh identifier?

### Bitcoin Tx Format

```
    nVersion = 0 or (FULL format only) as appropriate (e.g. 2, for time locks)
    nLockTime = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].prevout.hash = to_spend.txid
    vin[0].prevout.n = 0
    vin[0].nSequence = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].scriptWitness = message_signature
    vout[0].nValue = 0
    vout[0].scriptPubKey = p2sh_address (DID method specific identifier)
```

### Have TX1 with UTXO1 on Chain

In [76]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.helper import str_to_bytes

In [56]:
# Some previous tx_id
prev_tx=str_to_bytes("f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16")
# Index of output to spend
prev_index=0

tx_in = TxIn(prev_tx, prev_index)
tx_in.prev_index

0

In [57]:
tx_in.serialize()

b'61e9e1389e03583319cbf6536f506c57c4efda75fc387836d9b304695cf4814f\x00\x00\x00\x00\x00\xff\xff\xff\xff'

In [58]:
# Amount int the TX Out
amount = 200000

# Locking pay to witness pubkey script (You know the private key)
script_pubkey = private_key.point.p2wpkh_script()

tx_out = TxOut(amount, script_pubkey)

In [59]:
# We are pretending this tx has been approriately signed and broadcast to network
onchain_tx = Tx(0,[tx_in],[tx_out])

## Write DID to Bitcoin

Need to spend from a UTXO we control (UTX01) in a transaction that has a UTXO containing the script_pubkey of the DID. It may also contain additional UTXO specifying other verification methods.

Note: this address is unspendable because the redeem script contains and OP Return

In [41]:
print("did is : ", offline_did, "with address : ", script.address(), " represenation of script_pubkey : ", script.script_pubkey())

print("Assocaited redeem script is : ", script)

did is :  did:btcr:2:3AwfyyBr3gG1rYnkWY57CAbi5x1vQySCHZ with address :  3AwfyyBr3gG1rYnkWY57CAbi5x1vQySCHZ  represenation of script_pubkey :  OP_HASH160 657eb9c2b217a8a3871c72101ccd3ee28680b7b0 OP_EQUAL 
Assocaited redeem script is :  OP_RETURN db3de05d88d61ddaf123eb65db790c64bc9662bdd8d8c46559b5aaa97461018d 


In [61]:
prev_tx = onchain_tx.hash()
prev_index = 0

# Identify UTXO1 as the input
tx_in= TxIn(prev_tx, prev_index)

In [69]:
# Need a output you can spend!
amount = 2000
script_pubkey = private_key.point.p2wpkh_script()

tx_out0 = TxOut(amount,script_pubkey)

In [70]:
# Can never spend this output. But using it to create onchain whilst binding to document
amount = 0
script_pubkey = script.script_pubkey()

tx_out1 = TxOut(amount,script_pubkey)

In [78]:
creation_tx = Tx(0, [tx_in], [tx_out0,tx_out1])

## Sign Tx and Broadcast

In [ ]:
#